In [1]:
import sys
import pygame
from pygame.locals import *
import pymunk 

from pymunk.vec2d import Vec2d

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Darwin (64bit) [/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/libchipmunk.dylib]


In [2]:
from pymunk import pygame_util
import random
import math

In [3]:
def add_bullet(space):
    mass = 1
    radius = 10
    moment = pymunk.moment_for_circle(mass, 0, radius) # 1
    body = pymunk.Body(mass, moment) # 2
    body.position = 0, 200 # 3
    body.start_position = Vec2d(body.position)
    shape = pymunk.Circle(body, radius) # 4
    
    shape.elasticity = 0
    shape.friction = 0
    shape.collision_type = 1
    
    shape.color = pygame.color.THECOLORS["black"]
    
    space.add(body, shape) # 5
    return shape

In [4]:
def add_pendulum(space):
    
    # x and y coords for block's center
    center_x = 300
    center_y = 100
    
    # Add pendulum's block
    #block_mass = 10000    
    #block_moment = pymunk.moment_for_box(block_mass, (200,100))
    block_body = pymunk.Body()
    
    block_body.position = center_x, center_y
    block_shape = pymunk.Poly(block_body, [(center_x-100, center_y-50),(center_x-100, center_y+50),(center_x+100, center_y+50),(center_x+100, center_y-50)])
    block_shape.density = .002
    
    block_shape.elasticity = 0
    block_shape.friction = 0
    block_shape.collision_type = 2
    
    block_shape.color = pygame.color.THECOLORS["gray"]
    
    space.add(block_body, block_shape)    
    
    
    # Make pendulum hang from two strings
    pivot_point_1 = pymunk.Body(body_type = pymunk.Body.STATIC)
    pivot_point_1.position = (center_x, center_y+100)
    
    pivot_point_2 = pymunk.Body(body_type = pymunk.Body.STATIC)
    pivot_point_2.position = (center_x, center_y+100)
    
    joint_1 = pymunk.constraint.PinJoint(pivot_point_1, block_body, (center_x-100, center_y+100), (center_x-100,center_y+50))
    joint_2 = pymunk.constraint.PinJoint(pivot_point_2, block_body, (center_x+100, center_y+100), (center_x+100,center_y+50))
    
    space.add(joint_1, joint_2)
    
    return block_shape

In [5]:
def bullet_hits(arbiter, space, _):    
    '''
    This function is called after the bullet finishes transferring momentum to block (during post_solve)
    '''
    # Transfer mass of bullet to block
    space.bodies[1].mass += (space.bodies[0].mass)
    
    space.remove(space.shapes[0]) # remove the bullet shape
    space.remove(space.bodies[0]) # remove the bullet body


In [6]:
class simulation:
    def __init__(self):
        self.screen_width, self.screen_height = 800, 600
        
        self.max_height_reached = None
        self.max_potential_energy = None
    
    def start_simulation(self):
        pygame.init()
        self.myfont = pygame.font.SysFont('Comic Sans MS', 30)

        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        self.screen.set_alpha(None)
        
        
        pygame.display.set_caption("Ballistic Pendulum Simulation")
        self.clock = pygame.time.Clock()

        self.background = pygame.Surface(self.screen.get_size())
        self.background = self.background.convert()
        self.background.fill((255, 255, 255))

        self.space = pymunk.Space()
        self.space.gravity = 0, -900
        self.space.damping = .9

        self.bullet = add_bullet(self.space)
        self.block = add_pendulum(self.space)

        # Setup bullet-block collision callback function
        self.h = self.space.add_collision_handler(1, 2) # ...(COLLTYPE_BULLET, COLLTYPE_BLOCK)
        self.h.post_solve = bullet_hits

        self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)
        self.draw_options.constraint_color = pygame.color.THECOLORS["brown"]
        self.draw_options.collision_point_color = pygame.color.THECOLORS["red"] 


        self.space.shapes[0].body.apply_impulse_at_local_point((15000,0))

    def run_simulation(self, renderCounter):
        
        self.screen.blit(self.background, (0, 0))
        
        # One-time Calculations (when pendulum reaches max-height)
        if(self.block.body.kinetic_energy < 30):
            self.max_height_reached = self.block.body.position[1] - 100
            self.max_potential_energy = self.block.body.mass*900*(self.block.body.position[1] - 100)
        
        
        if(renderCounter >=5):
            # Ongoing Calculations
            self.text_h = self.myfont.render("Block Height: %d" % (self.block.body.position[1] - 100), True, (0, 0, 0), (255,255,255))
            self.screen.blit(self.text_h, (self.screen_width/2-self.text_h.get_rect().width/2, self.screen_height/2 - 200))

            self.text_ke = self.myfont.render("Block KE: %d" % (self.block.body.kinetic_energy), True, (0, 0, 0), (255,255,255))
            self.screen.blit(self.text_ke, (self.screen_width/2-self.text_ke.get_rect().width/2, self.screen_height/2 - 250))

            if(self.max_height_reached != None):
                self.text_maxh = self.myfont.render("Max Height Reached: %d" % self.max_height_reached, True, (0, 0, 0), (255,255,255))
                self.screen.blit(self.text_maxh, (self.screen_width/2-self.text_maxh.get_rect().width/2, self.screen_height/2 + 200))

                self.text_mgh = self.myfont.render("Max PE Reached: %d" % self.max_potential_energy, True, (0, 0, 0), (255,255,255))
                self.screen.blit(self.text_mgh, (self.screen_width/2-self.text_mgh.get_rect().width/2, self.screen_height/2 + 250))
                
                
            renderCounter = 0
            
        else:
            renderCounter +=1
            self.screen.blit(self.text_h, (self.screen_width/2-self.text_h.get_rect().width/2, self.screen_height/2 - 200))
            self.screen.blit(self.text_ke, (self.screen_width/2-self.text_ke.get_rect().width/2, self.screen_height/2 - 250))
            if(self.max_height_reached != None):
                self.screen.blit(self.text_maxh, (self.screen_width/2-self.text_maxh.get_rect().width/2, self.screen_height/2 + 200))
                self.screen.blit(self.text_mgh, (self.screen_width/2-self.text_mgh.get_rect().width/2, self.screen_height/2 + 250))
            
            
            
    
        self.space.step(1/500.0) # TEMP: Slow-motion
        
        self.space.debug_draw(self.draw_options)


        pygame.display.flip()
        self.clock.tick(50)
        
        return renderCounter
    
    def restart_simulation(self):
    
        numShapes = len(self.space.shapes)
        for i in range(numShapes):
            self.space.remove(self.space.shapes[0])
            
        numBodies = len(self.space.bodies)
        for i in range(numBodies):
            self.space.remove(self.space.bodies[0])      
        
        numConstraints = len(self.space.constraints)
        for i in range(numConstraints):
            self.space.remove(self.space.constraints[0])
        
        self.bullet = add_bullet(self.space)
        self.block = add_pendulum(self.space)
        
        self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)
        
        self.space.shapes[0].body.apply_impulse_at_local_point((15000,0))
        
        self.max_height_reached = None
        self.max_potential_energy = None
        
        
        self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)
        self.draw_options.constraint_color = pygame.color.THECOLORS["brown"]
        
        print("Restart complete") # TEMP
        
        
        

In [7]:
def main():
    pendulumSimulation = simulation()
    pendulumSimulation.start_simulation()
    
    pygame.event.set_allowed([QUIT, KEYDOWN])
    
    renderCounter = 5
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                sys.exit(0)
            elif event.type == KEYDOWN and event.key == K_ESCAPE:
                sys.exit(0)
            elif event.type == KEYDOWN and event.key == K_r:
                pendulumSimulation.restart_simulation()
                
            
        renderCounter = pendulumSimulation.run_simulation(renderCounter)
            

In [ ]:
if __name__ == '__main__':
    sys.exit(main())